In [2]:
#!M
import pandas as pd
import numpy as np
import gensim
import pymorphy2
import nltk
import os
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.models import FastText
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Импорт данных

In [3]:
#!M
df = pd.read_csv('data_banks.csv', 
                 sep = ';', 
                 quotechar='"', 
                 doublequote=False, 
                 error_bad_lines = False, 
                 escapechar ="'", 
                 encoding = 'UTF-8')
print(df.shape)
df.head()

b'Skipping line 510720: expected 21 fields, saw 22\n'


(563232, 21)


,id,load_number,spider_name,url,h1,title,product_type,client_type,bank,review_user,...,rating_value,review_datetime,review_header,review_body,rating_status,comments_count,views_count,answer_body,answer_datetime,answer_author
0,174329,3,banki,https://www.banki.ru/services/responses/bank/r...,NaN,Отзывы о Тинькофф Банке: «ЗАБЛОКИРОВАЛИ ВХОДЯЩ...,debitcards,NaN,Тинькофф Банк,user9598123,...,1,2020-09-08 13:22:58,ЗАБЛОКИРОВАЛИ ВХОДЯЩИЙ ПЛАТЕЖ,Добрый день! 01.09.2020 был осуществлен плате...,Проверяется,6.0,343,Приветствуем. Запрашиваем документы только при...,08.09.2020 16:44,Тинькофф Банк
1,174330,3,banki,https://www.banki.ru/services/responses/bank/r...,NaN,Отзывы о ОТП Банке: «Болото...» | Банки.ру,debitcards,NaN,ОТП Банк,v*******@gmail.com,...,1,2020-09-08 07:19:13,Болото...,Обратился за открытием карт-счета. В офисе Ом...,Проверяется,0.0,459,Уважаемый v*******@gmail.com! спасибо за Ваш о...,08.09.2020 8:03,ОТП Банк
2,174331,3,banki,https://www.banki.ru/services/responses/bank/r...,NaN,Отзывы о РНКБ: «Зарплатная карта» | Банки.ру,debitcards,NaN,РНКБ,l.khar,...,4,2020-09-07 18:53:28,Зарплатная карта,Не так давно на работе перевели на зарплатный...,Проверяется,0.0,306,Здравствуйте! Благодарим Вас за отзыв. Разрабо...,07.09.2020 19:27,РНКБ
3,174332,3,banki,https://www.banki.ru/services/responses/bank/r...,NaN,Отзывы о Сбербанке России: «Очень много навязы...,debitcards,NaN,Сбербанк России,perevorochaeva,...,3,2020-09-07 14:10:27,Очень много навязывают страховое и прочих услуг,Клиентом Сбербанка являюсь довольно давно (с ...,Проверяется,0.0,425,"Здравствуйте! Мне очень жаль, что посещение оф...",07.09.2020 18:50,Сбербанк России
4,174333,3,banki,https://www.banki.ru/services/responses/bank/r...,NaN,Отзывы о Московском Индустриальном Банке: «При...,debitcards,NaN,Московский Индустриальный Банк,Sereg64,...,2,2020-09-07 11:14:53,При замене неработоспособной карты пенсионера...,"У меня отзыв по карте пенсионера, которой я п...",Проверяется,1.0,287,Уважаемый Клиент! Благодарим Вас за обращение ...,09.09.2020 11:32,Московский Индустриальный Банк


# Предобработка данных

In [4]:
#!M
df = df[((df['spider_name']=='sravni') & (df['load_number']==2))  |  ((df['spider_name'] == 'banki') & (df['load_number']==3))]
df = df.drop_duplicates()
df= df.dropna(axis='index', how='any', subset=['review_body'])
df = df.reset_index()

In [5]:
#!M

from multiprocessing import  Pool
def all_prep_func(df):
    
    #Токенизация
    tokens = df['review_body'].apply(lambda x: gensim.utils.simple_preprocess(x, min_len=2, max_len=60))
    #Нормализация
    morph = pymorphy2.MorphAnalyzer()
    tks_norm = tokens.apply(lambda x: [morph.parse(i)[0].normal_form for i in x])
    #Удаляем стоп-слова
    stop_words = stopwords.words('russian')#!!!!!!!!!
    stop_words.extend(['это', 'весь', 'quot', 'который', 'ещё', 'свой', 'br', 'ваш', 'очень', 'почему', 'хотя', 'самый', 'сразу', 'вообще', 'банк','как','не','очень','это','на','банка','весь','по','что','но','быть','ни','один','всегда','нет','из','самый', 'сей','плач','естественно','далее', 'ттс','начало','отп','ndash','акб','ikea','поэтому','ситибанк','бки','совкомбанк','либо','девушка','видимо','например','ru','росбанк','мкб','январь','февраль','март','апрель','май','июнь','июль'])
    stop_words.remove('хорошо')
    stop_words.remove('лучше')
    tokens_norm_nosw = tks_norm.apply(lambda tks_norm_arr: [x for x in tks_norm_arr if x not in stop_words])
    #склеиваем в строку
    text_norm_nosw = tokens_norm_nosw.apply(lambda x: ' '.join(x))
    
    # Добавляем столбцы в датафрейм
    df['tokens_norm_nosw'] = tokens_norm_nosw
    df['text_norm_nosw'] = text_norm_nosw
    
    return df


def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df





In [10]:
#!XL
nltk.download('stopwords')
df_pp = parallelize_dataframe(df, all_prep_func, n_cores = 32)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
#!M
df_pp= df_pp.dropna(axis='index', how='any', subset=['tokens_norm_nosw'])
df_pp = df_pp.reset_index()

In [1]:
#!M
df_pp.to_csv('data_banks_pp.csv')

# Обучение FastText

In [14]:
#!M
model = FastText(sg = 1, 
                 size=300, 
                 window=10, 
                 min_count=50, 
                 min_n = 3, 
                 max_n = 3, 
                 workers = 32) #ПОСТАВИТЬ В СООТВЕТСТВИИ СО СРЕДОЙ ВЫПОЛНЕНИЯ
model.build_vocab(df_pp['tokens_norm_nosw'])

In [15]:
#!M
from gensim.test.utils import get_tmpfile

from gensim.models.callbacks import CallbackAny2Vec

class EpochLogger(CallbackAny2Vec):

    '''Callback to log information about training'''

    def __init__(self):

        self.epoch = 0


    def on_epoch_begin(self, model):

        print("Epoch #{} start".format(self.epoch))


    def on_epoch_end(self, model):

        print("Epoch #{} end".format(self.epoch))

        self.epoch += 1

In [17]:
#!XL
epoch_logger = EpochLogger()
model.train(sentences=df_pp['tokens_norm_nosw'], 
            total_examples=len(df_pp['tokens_norm_nosw']), 
            epochs=10, 
            callbacks=[epoch_logger])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end


## Сохраняем модель в файл


In [18]:
#!M
fname = os.path.join('saved_models', 'fasttext.model')
model.save(fname)